# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

In [1]:
# Installing the required libraries in the local venv

# !pip install xgboost
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install scikit-learn

### Install packages

In [2]:
# !pip install wandb -qU

In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [2]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [4]:
diamonds.shape

(53940, 10)

In [6]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [9]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

In [10]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 545.191877397669


/Users/johnny/Documents/GitHub/MSAI490_HW4/.venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Incorporate validation

In [11]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [12]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[10]	train-rmse:554.29819	validation-rmse:579.26422
[20]	train-rmse:493.68077	validation-rmse:547.75493
[30]	train-rmse:467.32713	validation-rmse:540.03567
[40]	train-rmse:447.40974	validation-rmse:541.70531
[50]	train-rmse:432.62075	validation-rmse:540.89769
[60]	train-rmse:422.28318	validation-rmse:540.63039
[70]	train-rmse:410.72350	validation-rmse:543.67077
[80]	train-rmse:398.24619	validation-rmse:545.08296
[90]	train-rmse:386.92486	validation-rmse:543.90036
[99]	train-rmse:379.58717	validation-rmse:545.19188


In [14]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[50]	train-rmse:432.62075	validation-rmse:540.89769
[83]	train-rmse:393.82435	validation-rmse:544.68591


In [15]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


In [16]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.512810,8.494816,2861.704341,37.144992
1,2081.847733,5.811005,2084.838207,31.889208
2,1547.031906,5.092391,1554.657450,30.699908
3,1184.129738,3.982239,1194.251600,26.940062
4,942.998782,3.327174,960.239319,24.392689


In [17]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

553.4613038243663

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [8]:
# Log in to your W&B account
import wandb

wandb.login()

/Users/johnny/Documents/GitHub/MSAI490_HW4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/johnny/Documents/GitHub/MSAI490_HW4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Currently logged in as: jhourmozdi (medtest). Use `wandb login --relogin` to force relogin


True

# Weights and Biases Experiment

**Assignment:**
- Start experiment tracking with W&B
- Do at least 5 experiments with various hyperparameters
- Choose any method for hyperparameter tuning: grid search, random search, bayesian search
- Describe your findings and what you see



Initially, I attempted to run XGBoost natively within a W&B run, and it output a very un-useful set of graphs which included only the parameters that I passed to W&B (RSME and learning rate for this first experiment), and then output a chart with the x-axis being "step" for both of these parameters, which you had to use to correlate the different learning rates with the different RSME achieved. 

By using W&B's sweep instead to do grid search, I was able to assess multiple different hyperparameters at once, and look at a number of differences in the model by "run" which were randomly named - including system level differences like CPU and memory utilization for the different hyperparameters. You can see the results in the screenshots below.

![learning rate](learning_rate.png) ![max_depth](max_depth.png) ![n_est](n_estimators.png) ![RSME](rsme.png) ![memory_util](process_mem.png)

## Code for Experiment

In [ ]:
# Load the diamonds dataset
diamonds = sns.load_dataset("diamonds")

# Preprocess the data
X = diamonds.drop(['price'], axis=1)
y = diamonds['price']

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.
X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the hyperparameter search space
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'rmse',
        'goal': 'minimize'
    },
    'parameters': {
        'max_depth': {
            'values': [3, 5, 7]
        },
        'n_estimators': {
            'values': [100, 200, 300]
        },
        'learning_rate': {
            'values': [0.01, 0.05, 0.1]
        }
    }
}

# Initialize the wandb sweep
sweep_id = wandb.sweep(sweep_config, project="xgboost-diamonds-2")

def train_model(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # Configure the XGBoost model with the hyperparameters
        config = wandb.config
        model = xgb.XGBRegressor(max_depth=config.max_depth,
                                  n_estimators=config.n_estimators,
                                  learning_rate=config.learning_rate,
                                  enable_categorical=True)

        # Train the model
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        rmse = ((y_pred - y_test) ** 2).mean() ** 0.5
        wandb.log({
            'learning_rate': config.learning_rate,
            'max_depth': config.max_depth,
            'n_estimators': config.n_estimators,
            'rmse': rmse})

# Run the sweep
wandb.agent(sweep_id, function=train_model, count=25)

Create sweep with ID: qyt00kph
Sweep URL: https://wandb.ai/medtest/xgboost-diamonds-2/sweeps/qyt00kph


wandb: Agent Starting Run: 7smaopun with config:
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 3
wandb: 	n_estimators: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/johnny/Documents/GitHub/MSAI490_HW4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


learning_rate,▁
max_depth,▁
n_estimators,▁
rmse,▁
learning_rate,0.01
max_depth,3
n_estimators,100
rmse,1831.91203


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 29h4t52z with config:
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 3
wandb: 	n_estimators: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/johnny/Documents/GitHub/MSAI490_HW4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


learning_rate,▁
max_depth,▁
n_estimators,▁
rmse,▁
learning_rate,0.01
max_depth,3
n_estimators,200
rmse,1126.62198


...

learning_rate,▁
max_depth,▁
n_estimators,▁
rmse,▁
learning_rate,0.1
max_depth,5
n_estimators,200
rmse,531.19823


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: tagrnkrr with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 5
wandb: 	n_estimators: 300
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/johnny/Documents/GitHub/MSAI490_HW4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


learning_rate,▁
max_depth,▁
n_estimators,▁
rmse,▁
learning_rate,0.1
max_depth,5
n_estimators,300
rmse,529.26351


wandb: Agent Starting Run: ihr6i8hb with config:
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 7
wandb: 	n_estimators: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/johnny/Documents/GitHub/MSAI490_HW4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


learning_rate,▁
max_depth,▁
n_estimators,▁
rmse,▁
learning_rate,0.1
max_depth,7
n_estimators,100
rmse,531.646


In [35]:
wandb.finish()